In [ ]:
import sys, os
repodir = os.path.abspath('..')
if not repodir in sys.path:
    sys.path.append(repodir)

In [ ]:
from aircleaning import html

In [ ]:
page = html.Page(
    html.Div(
        "Lorem ipsum dolor sit amet, consectetur adipiscing elit, ",
        html.Tooltip(
            "sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.",
            "Lorem ipsum dolor sit amet",
            ),
        " Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.",
        style="padding:16px",
        ),
    )
page.save_html('test')
display(page)

In [ ]:
from aircleaning.produce import *

soft = True

In [ ]:
width_range = range(4, 8)
length_range = range(4, 8)
height_range = range(2, 5)
window_range = range(6)
door_range = range(3)
mech_range = range(2)
person_range = range(1, 6, 1)
activity_range = range(3)
allvols = tuple(sorted(set(map(np.product, itertools.product(
    width_range, length_range, height_range,
    )))))
ach_range = range(0, 16)
cadrs = tuple(sorted(set(
    math.ceil(val / 100) * 100
    for val in map(np.product, tuple(itertools.product(ach_range, allvols)))))
    )

if not soft:
    outpath = os.path.join(productsdir, 'rooms')
    room_combos = tuple(itertools.product(
        width_range, length_range, height_range,
        person_range, activity_range,
        window_range, door_range, mech_range,
        ))
    for room_combo in room_combos:
        scale = np.power(1/np.product(room_combo[:3]), 1/3) / np.power(1/np.min(allvols), 1/3)
        room = svg.draw_scene(*room_combo, size=2)
        room.projection.scale(scale)
        room.save_svg('_'.join(map(str, room_combo)), outpath)

if not soft:
    outpath = os.path.join(productsdir, 'costs')
    for cadr in cadrs:
        cost_analysis_by_cadr(max(100, cadr), path=outpath, name=str(cadr))

science_code = html.Script(
    '''function calculate_natural_vent_rate(windows, doors){''',
    '''  const wind_speed = 3;''',
    '''  const window_area = 0.5 * 0.8 * windows;''',
    '''  const door_area = 0.6 * 1.9 * doors;''',
    '''  if (windows>0 && doors>0) {''',
    '''      var coeff = 0.1;''',
    '''      var aperture = Math.min(window_area, door_area);''',
    '''    } else {''',
    '''      var coeff = 0.01;''',
    '''      var aperture = Math.max(window_area, door_area);''',
    '''    };''',
    '''  return coeff * wind_speed * aperture * 3600;''',
    '''  };''',
    '''function calculate_mech_vent_rate(mech, volume){''',
    '''  return mech * 6 * volume;''',
    '''  };''',
    '''function calculate_vent_rate(windows, doors, mech, volume){''',
    '''  const natural = calculate_natural_vent_rate(windows, doors);''',
    '''  const mechanical = calculate_mech_vent_rate(mech, volume);''',
    '''  const baseline = 3 * volume;''',
    '''  return natural + mechanical + baseline''',
    '''  };''',
    '''function calculate_fouling_rate(persons, activity, volume){''',
    '''  const multiplier = [1, 3, 10][activity];''',
    '''  const policy_1 = 0.35 * multiplier * volume * persons;''',
    '''  const l_s_pp = 10;''',
    '''  const m3_h_pp = l_s_pp * 3600 / 1000;''',
    '''  const policy_2 = multiplier * m3_h_pp * persons;''',
    '''  return Math.max(policy_1, policy_2)''',
    '''  };''',
    )

form_update = html.Script(
    '''function form_update(form){''',
    '''  const productsdir = "https://rsbyrne.github.io/aircleaning/products";''',
    '''  const length = form.room_length.value;''',
    '''  const width = form.room_width.value;''',
    '''  const height = form.room_height.value;''',
    '''  const windows = form.number_windows.value;''',
    '''  const doors = form.number_doors.value;''',
    '''  const persons = form.number_people.value;''',
    '''  const room = document.getElementById('room_viz');''',
    '''  const mech = form.mech_vent.value;''',
    '''  const activity = parseInt(form.activity_level.value);''',
    '''  room.src =''',
    '''    productsdir + "/rooms/" +''',
    '''    [length, width, height, persons, activity, windows, doors, mech].join("_")''',
    '''    + ".svg";''',
    '''  const volume = form.room_length.value * form.room_width.value * form.room_height.value;''',
    '''  document.getElementById("total_volume").innerHTML = volume;''',
    '''  const vent_rate = calculate_vent_rate(windows, doors, mech, volume);''',
    '''  const provided_cleaning = Math.round(vent_rate / volume);''',
    '''  document.getElementById("provided_cleaning").innerHTML = provided_cleaning;''',
    '''  const fouling_rate = calculate_fouling_rate(persons, activity, volume);''',
    '''  const required_cleaning = Math.max(4, Math.round(fouling_rate / volume));''',
    '''  document.getElementById("required_cleaning").innerHTML = required_cleaning;''',
    '''  const extra_cleaning = Math.max(0, required_cleaning - provided_cleaning);''',
    '''  document.getElementById("extra_cleaning").innerHTML = extra_cleaning;''',
    '''  const extra_cleaning_cadr = Math.ceil(Math.min(15, extra_cleaning) * volume / 100) * 100;''',
    '''  document.getElementById("extra_cleaning_cadr").innerHTML = extra_cleaning_cadr;''',
    '''  const acchart = document.getElementById("air_cleaner_recommendations");''',
    '''  acchart.src =''',
    '''    productsdir + "/costs/" +''',
    '''    Math.max(100, extra_cleaning_cadr)''',
    '''    + ".png";''',
    '''  }''',
    )

all_style = html.Style(
    # '''.container {''',
    # '''  max-width: 940px;''',
    # '''  margin: 0 auto;''',
    # '''  display: grid;''',
    # '''  grid-template-columns: 1fr 3fr;''',
    # '''  grid-gap: 10px;''',
    # '''}''',
    '''.solid {''',
    '''  color: black;'''
    '''  background-color: white;''',
    '''  };''',
    '''.input-fieldset {''',
    '''  };''',
    )

room_fields = html.Fieldset(
    'How big is your room?',
    {
        'Length (m):': html.CapturedInput(
            'range', 5, input_kwargs=dict(
                min=length_range.start, max=length_range.stop-1, step=length_range.step,
                instyles="width: 90%", name='room_length',
                ),
            ),
        'Width (m):': html.CapturedInput(
            'range', 4, input_kwargs=dict(
                min=width_range.start, max=width_range.stop-1, step=width_range.step,
                instyles="width: 90%", name='room_width',
                )
            ),
        'Height (m):': html.CapturedInput(
            'range', 2, input_kwargs=dict(
                min=height_range.start, max=height_range.stop-1, step=height_range.step,
                instyles="width: 90%", name='room_height',
                ),
            ),
        },
    html.Div(
        html.Div(
            "Total volume (m<sup>3</sup>):",
            instyles="float: left; width:70%; margin-right:10px",
            ),
        html.Div('-', identity='total_volume', instyles=("float: left", "width:20%")),
        instyles="margin-upper:10px"
        ),
    name='room_size',
    title="Dimensions",
    classes="input-fieldset",
    # instyles="margin: 10px;",
    )

usage_fields = html.Fieldset(
    "How is the room being used?",
    {
        'Number of people:': html.CapturedInput(
            'range', 1, input_kwargs=dict(
                min=person_range.start, max=person_range.stop-1, step=person_range.step,
                instyles="width: 90%", name='number_people',
                )
            ),
        'Level of activity:': html.Div(
            html.Div(*(
                html.LabelledInput(
                    level,
                    html.Input('radio', i, checked=i==0, name='activity_level'),
                    instyles=("padding-left:10px","padding-right:10px"),
                    )
                for i, level in enumerate(('Relaxed', 'Moderate', 'Intense'))
                ), instyles=("display:flex","justify-content:center","align-items:center")),
            instyles="margin:10px",
            ),
        },
    html.Div(
        html.Div(
            "Required cleaning (ACH):",
            instyles=("float: left","width:70%","margin-right:10px"),
            ),
        html.Div('-', identity='required_cleaning', instyles=("float: left","width:20%")),
        instyles="margin-upper:10px"
        ),
    name='room_usage',
    title="Usage",
    classes="input-fieldset",
    # instyles="margin: 10px;",
    )

cleaning_fields = html.Fieldset(
    html.Div(
        "How much cleaning is already available?",
        html.Tooltip(
            "(?)",
            "This is a tooltip.",
            ),
        ),
    {
        'Number of open windows:': html.CapturedInput(
            'range', 0, input_kwargs=dict(
                min=window_range.start, max=window_range.stop-1, step=window_range.step,
                instyles="width: 90%", name='number_windows',
                ),
            ),
        'Number of open doors:': html.CapturedInput(
            'range', 0, input_kwargs=dict(
                min=door_range.start, max=door_range.stop-1, step=door_range.step,
                instyles="width: 90%", name='number_doors',
                ),
            ),
        'Mechanical ventilation:': html.LabelledInput(
            'Yes', html.Input('checkbox', 0, name='mech_vent'),
            instyles=("display:flex","justify-content:center","align-items:center"),
            ),
        },
    html.Div(
        html.Div(
            "Provided cleaning (ACH):",
            instyles=("float: left", "width:70%", "margin-right:10px"),
            ),
        html.Div('-', identity='provided_cleaning', instyles=("float: left", "width:20%")),
        instyles="margin-upper:10px"
        ),
    name='room_cleaning',
    title="Circulation",
    classes="input-fieldset",
    # instyles="margin: 10px;",
    )

room_viz = html.Div(
    html.Image(
        "https://via.placeholder.com/150", identity="room_viz",
        instyles=("max-width:100%", "max-height:100%")
        ),
    # instyles="float: right; width: 50%; margin: 10px;",
    )

input_selector = html.TabbedPanes(
    room_fields,
    usage_fields,
    cleaning_fields,
    identity="input_selector",
    # instyles="width: 45%; float: left; text-align:center;",
    )

input_form = html.Form(
    input_selector,
    identity='user_form',
    )

input_section = html.Div(
    input_form,
    room_viz,
    instyles=(
        '''display: grid''',
        '''grid-template-columns: 60% 40%''',
        '''overflow:auto''',
        )
    # style="width:100%;"
    # style="display:flex;justify-content:center; margin:10px; width:100%",
    # style="display:flex;justify-content:center;align-items:center; margin:10px",
    )

summary_widget = html.Div(
    html.Div(
        "<b>Extra cleaning required:</b>",
        instyles=("display:flex","justify-items:center")
        ),
    html.Div(
        html.Div(
            html.Div(
                "<i>Air changes (ACH):</i>",
                ),
            html.Div(
                '-',
                identity='extra_cleaning',
                ),
            instyles=(
                '''display: grid''',
                '''grid-template-columns: 1fr 1fr''',
                '''justify-items:center''',
                ),
            ),
        html.Div(
            html.Div(
                "<i>CADR (m<sup>3</sup>/hr):</i>",
                ),
            html.Div(
                '-',
                identity='extra_cleaning_cadr',
                ),
            instyles=(
                '''display: grid''',
                '''grid-template-columns: 1fr 1fr''',
                '''justify-items:center''',
                ),
            ),
        instyles=(
            '''display: grid''',
            '''grid-template-rows: 1fr 1fr''',
            ),
        ),
    instyles=(
        '''display: grid''',
        '''grid-template-rows: 30px 1fr''',
        '''justify-items: center''',
        ),
    )

summary_section = html.Div(
    summary_widget,
    )

tool_selector = html.TabbedPanes(
    html.Image(
        '-',
        identity="air_cleaner_recommendations",
        instyles=(
            "display:flex", "justify-content:center",
            "align-items:center", "margin:10px",
            ),
        title="Air cleaners",
        ),
    # html.Image(
    #     "https://via.placeholder.com/150",
    #     instyles="display:flex;justify-content:center;align-items:center; margin:10px",
    #     title="Air quality",
    #     ),
    # html.Image(
    #     "https://via.placeholder.com/150",
    #     instyles="display:flex;justify-content:center;align-items:center; margin:10px",
    #     title="Infection risk",
    #     ),
    identity="tool_selector",
    # instyles="width: 100%",
    )

output_section = html.Div(
    tool_selector,
    instyles=("display: grid", "grid-template-rows: 1fr 3fr"),
    )

all_content = html.Div(
    input_section,
    summary_section,
    output_section,
    instyles=(
        '''display: grid''',
        '''grid-template-rows: 25% 7% 68%''',
        '''max-height: 11.7in''',
        '''max-width: 8.3in''',
        )
    # classes=('container',)
    )

initialise_script = html.Script(
    '''form_update(document.getElementById("user_form"));''',
    '''document.getElementById("tool_selector_button_0").click();''',
    '''document.getElementById("input_selector_button_0").click();''',
    )

page = html.Page(
    # prep_code,
    science_code,
    form_update,
    all_style,
    html.Div(
        all_content,
        instyles=(
            '''display: flex''',
            '''justify-items: center''',
            '''flex-wrap: wrap''',
            '''align-items: center''',
            '''justify-content: center''',
            ),
        classes=('solid',)
        ),
    initialise_script,
    # instyles="float:center;margin:auto",
    )

display(page)
page.save_html('test')

In [ ]:
room_viz.instyles

In [ ]:
# page = html.Page(
#     html.Style(
#         '''/* Tooltip container */''',
#         '''.tooltip {''',
#         '''  position: relative;''',
#         '''  display: inline-block;''',
#         '''  border-bottom: 1px dotted black; /* If you want dots under the hoverable text */''',
#         '''}''',
#         '''/* Tooltip text */''',
#         '''.tooltip .tooltiptext {''',
#         '''  visibility: hidden;''',
#         '''  width: 120px;''',
#         '''  top: 100%;''',
#         '''  left: 50%;''',
#         '''  margin-left: -60px; /* Use half of the width (120/2 = 60), to center the tooltip */''',
#         '''  color: black;''',
#         '''  background-color: #FFDCA2;''',
#         '''  text-align: center;''',
#         '''  padding: 5px 0;''',
#         '''  border-radius: 6px;''',
#         '''  /* Position the tooltip text - see examples below! */''',
#         '''  position: absolute;''',
#         '''  z-index: 1;''',
#         '''}''',
#         '''/* Show the tooltip text when you mouse over the tooltip container */''',
#         '''.tooltip:hover .tooltiptext {''',
#         '''  visibility: visible;''',
#         '''}''',
#         '''/* Add an arrow */''',
#         '''.tooltip .tooltiptext::after {''',
#         '''  content: " ";''',
#         '''  position: absolute;''',
#         '''  bottom: 100%;  /* At the top of the tooltip */''',
#         '''  left: 50%;''',
#         '''  margin-left: -5px;''',
#         '''  border-width: 5px;''',
#         '''  border-style: solid;''',
#         '''  border-color: transparent transparent #FFDCA2 transparent;''',
#         '''}''',
#         '''/* Fade in the tooltip text */''',
#         '''.tooltip .tooltiptext {''',
#         '''  opacity: 0;''',
#         '''  transition: opacity 0.5s;''',
#         '''}''',
#         '''.tooltip:hover .tooltiptext {''',
#         '''  opacity: 1;''',
#         '''}''',
#         ),
#     html.Div(
#         '''Hover over me''',
#         html.Span(
#             '''Tooltip text''',
#             classes=("tooltiptext",),
#             ),
#         classes=('tooltip',),
#         ),
#     )

# page.save_html('test')
# page

# # page = html.Page(
# #     html.Div(
# #         '''<h1>Wikipedia Tooltip.</h1>''',
# #         html.Paragraph(
# #             '''Lorem ipsum dolor sit amet consectetur adipisicing elit. Voluptas, harum ''',
# #             html.Span(
# #                 html.Hyperlink('#', '''Tooltip 1'''),
# #                 tooltipID='1',
# #                 classes=('tooltip-originator',),
# #                 ),
# #             ''' rem dolorem impedit sapiente aliquam quidem ut voluptates dolorum?''',
# #             ),
# #         html.Div(classes=("tooltip-destination",)),
# #         ),
# #     html.Style(
# #         '''*''',
# #         # '''::before,''',
# #         # '''::after {''',
# #         # '''  box-sizing: border-box;''',
# #         # '''  margin: 0;''',
# #         # '''  padding: 0;''',
# #         # '''  }''',
# #         # '''body {''',
# #         # '''  font-family: Arial, Helvetica, sans-serif;''',
# #         # '''  overflow-x: hidden;''',
# #         # '''  }''',
# #         # '''.container {''',
# #         # '''  padding: 50px 10vw 100px;''',
# #         # '''  font-size: 40px;''',
# #         # '''  }''',
# #         # '''a:visited {color:#0645ad}''',
# #         # '''''',
# #         # '''.container h1 {''',
# #         # '''  margin-bottom: 20px;''',
# #         # '''  }''',
# #         '''.tooltip-destination {''',
# #         '''  width: 425px;'''
# #         '''  min-height: 200px;''',
# #         '''  height: auto;''',
# #         '''  padding: 15px;''',
# #         '''  font-size: 25px;''',
# #         '''  background: white;''',
# #         '''  box-shadow: 0 30px 90px -20px rgba(0,0,0,0.3);''',
# #         '''  position: absolute;''',
# #         '''  z-index: 100;'''
# #         '''  display: none;'''
# #         '''  opacity: 0;''',
# #         '''  }''',
# #         '''.tooltip-fade-in {''',
# #         '''  display: block;''',
# #         '''  animation: tooltip-fade 0.2s linear forwards;''',
# #         '''  }''',
# #         '''@keyframes tooltip-fade {''',
# #         '''  0% {''',
# #         '''    opacity: 0;''',
# #         '''    }'''
# #         '''  100% {''',
# #         '''    opacity: 1;''',
# #         '''    }''',
# #         '''  }''',
# #         ),
# #     html.Script(
# #         '''function initialise_tooltips(){'''
# #         '''  const tooltips = Array.from(document.querySelectorAll(".tooltip-originator"));''',
# #         '''  const tooltipContainer = document.querySelector(".tooltip-destination");''',
# #         '''  const data = [''',
# #         '''    {id: 1, txt: "Lorem ipsum dolor sit amet consectetur adipisicing elit. Atque possimus assumenda quis illo minus numquam voluptates nihil, doloremque unde non."},''',
# #         # '''    {id: 2, txt: "Lorem ipsum, dolor sit amet consectetur adipisicing elit. Aliquid, deleniti."},''',
# #         # '''    {id: 3, txt: "Lorem ipsum dolor sit amet."}''',
# #         '''    ]''',
# #         '''  let tooltipID;''',
# #         '''  tooltips.forEach((tooltip) => {''',
# #         '''    tooltip.addEventListener("mouseenter", (e) => {''',
# #         '''      tooltipID = e.target.getAttribute('tooltipID');''',
# #         '''      tooltipContainer.classList.add("tooltip-fade-in");''',
# #         '''      tooltipContainer.style.left = `${e.pageX}px`;''',
# #         '''      tooltipContainer.style.top = `${e.pageY}px`;''',
# #         '''      tooltipContainer.innerText = data[tooltipID - 1].txt;''',
# #         '''      });''',
# #         '''    tooltip.addEventListener("mouseout", (e) => {''',
# #         '''      tooltipContainer.classList.remove("tooltip-fade-in");''',
# #         '''      });''',
# #         '''    });''',
# #         '''  tooltipContainer.addEventListener('mouseenter', () => {''',
# #         '''    tooltipContainer.classList.add("tooltip-fade-in");''',
# #         '''    })''',
# #         '''  tooltipContainer.addEventListener('mouseout', () => {''',
# #         '''    tooltipContainer.classList.remove("tooltip-fade-in");''',
# #         '''    })''',
# #         '''  }''',
# #         '''initialise_tooltips()'''
# #         ),
# #     )
# # page.save_html('test')
# # page